In [1]:
import os
import gc
import sys
import glob
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import keras
import tensorflow as tf
from keras.utils import plot_model
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint, CSVLogger

sys.path.insert(0, "../../python")
from common import *


Using TensorFlow backend.


In [2]:
class NetSetup:
    def __init__(self, activation, dropout_rate, first_layer_size, last_layer_size, decay_factor, kernel_regularizer):
        self.activation = activation
        if activation == 'relu':
            self.DropoutType = Dropout
            self.kernel_init = 'he_uniform'
            self.apply_batch_norm = True
        elif activation == 'selu':
            self.DropoutType = AlphaDropout
            self.kernel_init = 'lecun_normal'
            self.apply_batch_norm = False
        else:
            raise RuntimeError('Activation "{}" not supported.'.format(activation))
        self.dropout_rate = dropout_rate
        self.first_layer_size = first_layer_size
        self.last_layer_size = last_layer_size
        self.decay_factor = decay_factor
        self.kernel_regularizer = kernel_regularizer
    
def add_block_ending(net_setup, name_format, layer):
    if net_setup.apply_batch_norm:
        norm_layer = BatchNormalization(name=name_format.format('norm'))(layer)
    else:
        norm_layer = layer
    activation_layer = Activation(net_setup.activation, name=name_format.format('activation'))(norm_layer)
    if net_setup.dropout_rate > 0:
        return net_setup.DropoutType(net_setup.dropout_rate, name=name_format.format('dropout'))(activation_layer)
    return activation_layer

def conv_block(prev_layer, filters, kernel_size, net_setup, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer=net_setup.kernel_init)(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), conv)

def reduce_n_features_2d(input_layer, net_setup, block_name):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

def create_model():
    comp_net_setup = NetSetup('relu', 0.25, 128, 16, 1.4, None)
    dense_net_setup = NetSetup('relu', 0, 512, 32, 1.4, keras.regularizers.l1(1e-5))
            
    model_name = "DeepTau2017v2p2"
    input_layer_tau = Input(name="input_tau", shape=(len(input_tau_branches),))
    input_layers = [ input_layer_tau ]
    
    high_level_features = [ input_layer_tau ]
    for loc in cell_locations:
        reduced_inputs = []
        for comp_id in range(len(component_names)):
            comp_name = component_names[comp_id]
            comp_branches = component_branches[comp_id]
            input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                     shape=(n_cells_eta[loc], n_cells_phi[loc], len(comp_branches)))
            input_layers.append(input_layer_comp)
            reduced_comp = reduce_n_features_2d(input_layer_comp, comp_net_setup, "{}_{}".format(loc, comp_name))
            reduced_inputs.append(reduced_comp)
        conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
        cell_output_size = 32
        prev_layer = conv_block(conv_all_start, cell_output_size, (1, 1), comp_net_setup, "{}_all".format(loc), 1)
        window_size = 4
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    prev_layer = Concatenate(name="features_concat")(high_level_features)
    current_size = dense_net_setup.first_layer_size
    n = 1
    while True:
        dense_layer = Dense(current_size, name="final_dense_{}".format(n),
                            kernel_initializer=dense_net_setup.kernel_init,
                            kernel_regularizer=dense_net_setup.kernel_regularizer)(prev_layer)
        prev_layer = add_block_ending(dense_net_setup, 'final_{{}}_{}'.format(n), dense_layer)
        n += 1
        if current_size == dense_net_setup.last_layer_size: break
        current_size = max(dense_net_setup.last_layer_size, int(current_size / dense_net_setup.decay_factor))

    output_layer = Dense(n_outputs, name="final_dense_{}".format(n), kernel_initializer=dense_net_setup.kernel_init)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model, model_name

In [3]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = ["accuracy", TauLosses.tau_crossentropy, TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet, TauLosses.sLe, TauLosses.sLmu, TauLosses.sLjet ]
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [4]:
TauLosses.SetSFs(1, 1, 1)
model, model_name = create_model()
compile_model(model, 1e-3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_inner_pfCand (InputLayer) (None, 21, 21, 36)   0                                            
__________________________________________________________________________________________________
input_inner_ele (InputLayer)    (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
input_inner_muon (InputLayer)   (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
inner_pfCand_conv_1 (Conv2D)    (None, 21, 21, 128)  4736        input_inner_pfCand[0][0]         
__________________________________________________________________________________________________
inner_ele_

In [8]:
plot_model(model, to_file='model.png', show_shapes=True)

In [2]:
n_hidden_layers = 20
n_neurons = 1024
input_shape=(129,)
n_outputs=4
model_name = "{}L{}N".format(n_hidden_layers, n_neurons)
model = Sequential()
model.add(Dense(n_neurons, input_shape=input_shape, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
for n in range(n_hidden_layers):
    model.add(Dense(n_neurons, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
model.add(Dense(n_outputs))
model.add(Activation("softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              133120    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
__________

In [3]:
21172228 / 791617

26.745544878394476